# Работа с расчётными сетками в МКЭ-библиотеке `scikit-fem`

## Библиотека `scikit-fem`
Библиотека [`scikit-fem`](https://github.com/kinnala/scikit-fem) – python-библиотека для метода конечных элементов (МКЭ). Она позволяет собирать разреженные матрицы из билинейных форм, а также вектора из линейных форм. Поддерживаются неструктурированные треугольные, четырёхугольные, тетраэдральные и гексаэдральные сетки.

Для установки библиотеки можно использовать следующую команду:
```
pip install scikit-fem[all]
```
Параметр `[all]` указывает, что нужно установить вспомогательные библиотеки `meshio` и `matplotlib`.

Основной модуль библиотеки – `skfem`, для краткости будем загружать его под псевдонимом `fem`. Также для удобства нам потребуется `numpy` с псевдонимом `np`.

In [ ]:
import numpy as np
import skfem as fem

## Расчётные сетки в `scikit-fem`
В библиотеке `scikit-fem` используется свой собственный класс для работы с сеткой.

Создадим самую простую треугольную сетку для квадратной области:

In [ ]:
mesh = fem.MeshTri()

Можно вывести общую информацию о сетке на экран с помощью команды `print(mesh)`, а отобразить сетку можно с помощью метода `mesh.draw()`:

In [ ]:
print(mesh)
mesh.draw()

В этой сетке всего два элемента (треугольника) и 4 вершины.

Мы можем увидеть список координат вершин в аттрибуте `p` и список топологических связей для элементов сетки в аттрибуте `t`.

В списке координат первая строка – x-координаты вершин, а вторая – y-координаты:

In [ ]:
mesh.p

В списке топологических связей каждый столбец перечисляет номера вершин, которые образуют очередной элемент:

In [ ]:
mesh.t

Попробуйте разные типы сеток: `MeshTri`, `MeshQuad`, `MeshTet`, `MeshHex`. Также можно использовать сетки с ячейками второго порядка:  `MeshTri2`, `MeshQuad2`, `MeshTet2`, `MeshHex2`

In [ ]:
m = fem.MeshTri()
print(m)
print("Nodes:")
print(m.p)
print("Elements:")
print(m.t)
m.draw()

### Треугольные сетки
По умолчанию `MeshTri` создаёт сетку из двух треугольников. Для удобства есть несколько вспомогательных функций для создания чуть более сложных сеток.

`MeshTri.init_symmetric()` создаст симметричную сетку:

In [ ]:
fem.MeshTri.init_symmetric().draw()

`MeshTri.init_sqsymmetric()` создаст сетку "Union Jack":

In [ ]:
fem.MeshTri.init_sqsymmetric().draw()

`MeshTri.init_lshaped()` создаст сетку для буквы "L":

In [ ]:
fem.MeshTri.init_lshaped().draw()

`MeshTri.init_circle(n)` создаст сетку для круга с уровнем измельчения `n`:

In [ ]:
fem.MeshTri.init_circle(2).draw()

`MeshTri.init_tensor(x, y)` создаст структурированную сетку с координатами `x` и `y`:

In [ ]:
fem.MeshTri.init_tensor(np.linspace(0, 1, 10), np.linspace(0, 1, 5)).draw()

Также сетку можно создать из списка с координатами вершин и списка топологический связей:

In [ ]:
m = fem.MeshTri([[1,-1,0, 0],
                 [0, 0,1,-1]],
                [[0, 0],
                 [1, 1],
                 [2, 3]])
m.draw(node_numbering=True, element_numbering=True)

Сетки можно сохранять и загружать с помощью методов `save` и `load`.

In [ ]:
mesh = fem.MeshTri.load("unstructured.msh")
mesh.draw()

## Преобразование сеток
У сеточных классов есть вспомогательные методы, которые применяют к сетке некоторую операцию и возвращают новую сетку в качестве результата.

Например, метод `refined(n)` возвращает измельчённую на `n` уровней сетку. Каждый уровень измельчения уменьшает размеры элементов в два раза. Измельчение на `n` уровней уменьшает размеры в $2^n$ раз.

In [ ]:
mesh = fem.MeshTri()
refined = mesh.refined(3)
refined.draw()

Метод `scaled(s)` умножает координаты на множитель `s`. Множитель `s` можно указать отдельно для каждой координаты.

In [ ]:
fem.MeshTri().scaled([2, 1]).draw()

Метод `translated(u)` сдвигает координаты на смещение `u`.

In [ ]:
m1 = fem.MeshTri()                    # сетка в единичном квадрате
m2 = fem.MeshTri().translated([1, 0]) # сетка сдвинута вправо на 1
mesh = m1 + m2 # Сетки можно объединять с помощью оператора "+"
mesh.draw()

Метод `morphed(xfunc, yfunc)` применяет отображение координат с помощью функций `xfunc(p)` и `yfunc(p)`. В функцию передаются координаты точек `p`.

In [ ]:
square_mesh = fem.MeshTri().refined(3)
radial_mesh = square_mesh.morphed(lambda p: (1+p[0])*np.cos(p[1]*np.pi/2),
                                  lambda p: (1+p[0])*np.sin(p[1]*np.pi/2))
radial_mesh.draw()

Метод `mirrored(normal, center)` применяет отражение относительно прямой с нормалью `normal` и центром в `center`. По умолчанию `center` в начале координат.

In [ ]:
fem.MeshTri.init_lshaped().mirrored([1,0]).draw()

Метод `smoothed` применяет сглаживание к внутренним вершинам сетки.

In [ ]:
fem.MeshTri.init_lshaped().refined(2).smoothed().draw()

## Локальное и иерархическое измельчение сетки
У метода `refined` для треугольных (и тетраэдральных) сеток есть возможность явно указать список элементов, которые нужно измельчить. В этом случае происходит измельчение выбранных элементов и возможно их соседей для сохранения конформности. 

In [ ]:
mesh = fem.MeshTri().refined(1)
mesh.draw(element_numbering=True)
refined_mesh = mesh.refined([1,2])
refined_mesh.draw(element_numbering=True)

Можно воспользоваться вспомогательным методом `elements_satisfying(test_func)`, где функция `test_func(p)` возвращает `True` для центров `p` нужных элементов.

In [ ]:
def in_lower_right(p):
    # Здесь p -- массив 2xN с координатами центров N элементов
    return (p[0] > 0.5) * (p[1] < 0.5)  # аналог x > 0.5 и y < 0.5

mesh = fem.MeshTri().refined(2)
mesh.draw(element_numbering=True)
elements = mesh.elements_satisfying(in_lower_right)
print(elements)

Можно организовать иерархическое измельчение, применяя метод `refined` несколько раз подряд с выбором новых элементов на каждом уровне.

In [ ]:
mesh = fem.MeshTri().refined(1)
for k in range(3): # три уровня измельчения
    elements = mesh.elements_satisfying(in_lower_right) # выбираем элементы в нижней правой четверти
    mesh = mesh.refined(elements) # локально измельчаем выбранные элементы

mesh.draw()

Использование метода `elements_satisfying` допустимо, когда выбор элементов возможен по координатам центров треугольников. На практике проверки только центра элемента может быть недостаточно.

Пусть мы теперь хотим сгустить сетку к прямой y=0.5. Воспользуемся наивным подходом:

In [ ]:
def on_horizontal_line(p):
    # Здесь p -- массив 2xN с координатами центров N элементов
    return p[1] == 0.5  # y == 0.5

mesh = fem.MeshTri().refined(1)
for k in range(3): # три уровня измельчения
    elements = mesh.elements_satisfying(on_horizontal_line) # выбираем элементы в нижней правой четверти
    mesh = mesh.refined(elements) # локально измельчаем выбранные элементы

mesh.draw()

Ни один из треугольников не был разбит, так как центры треугольников в данном случае всегда либо выше, либо ниже центральной линии.

Вместо центров треугольников мы можем проверять вершины треугольников. Воспользуемся аттрибутами `p` и `t` с координатами вершин и номерами вершин в треугольниках.

In [ ]:
mesh = fem.MeshTri().refined(1)
for k in range(5): # пять уровней измельчения
    elements = [] # начнём с пустого списка элементов для измельчения

    for i in range(mesh.nelements): # mesh.nelements – количество элементов
        node_indices = mesh.t[:, i]  # Номера трех вершин треугольника с номером i (массив из 3 чисел)
        triangle_nodes = mesh.p[:, node_indices] # массив координат из трёх вершин (массив 2x3)
        triangle_nodes_y = triangle_nodes[1, :]  # y-координаты трёх вершин (массив из 3 чисел)
        if (np.any(triangle_nodes_y == 0.5)):    # если хотя бы одна из y-координат равна 0.5
            elements.append(i)

    mesh = mesh.refined(elements)

mesh.draw()

Знатоки Python могут переписать это в более компактном виде:

In [ ]:
mesh = fem.MeshTri().refined(1)
for k in range(5): # пять уровней измельчения
    elements = np.any(mesh.p[1, mesh.t] == 0.5, axis=0)
    mesh = mesh.refined(elements)

mesh.draw()

Приведённый выше метод с проверкой вершин на центральной линии работает, так как изначально в нашей сетке уже есть вершины на прямой y=0.5. Однако этот метод не сработает для прямой y=0.6:

In [ ]:
mesh = fem.MeshTri().refined(1)
for k in range(5): # пять уровней измельчения
    elements = [] # начнём с пустого списка элементов для измельчения

    for i in range(mesh.nelements): # mesh.nelements – количество элементов
        node_indices = mesh.t[:, i]  # Номера трех вершин треугольника с номером i (массив из 3 чисел)
        triangle_nodes = mesh.p[:, node_indices] # массив координат из трёх вершин (массив 2x3)
        triangle_nodes_y = triangle_nodes[1, :]  # y-координаты трёх вершин (массив из 3 чисел)
        if np.any(triangle_nodes_y == 0.6):    # если хотя бы одна из y-координат равна 0.5
            elements.append(i)

    mesh = mesh.refined(elements)

mesh.draw()

Нужно модифицировать проверку так, чтобы определять пересечение треугольника с прямой y=0.6. У таких треугольников хотя бы одна из вершин находится выше прямой, и хотя бы одна ниже. Изменим наш код, чтобы проверить это условие.

In [ ]:
mesh = fem.MeshTri().refined(1)
for k in range(5): # пять уровней измельчения
    elements = [] # начнём с пустого списка элементов для измельчения

    for i in range(mesh.nelements): # mesh.nelements – количество элементов
        node_indices = mesh.t[:, i]  # Номера трех вершин треугольника с номером i (массив из 3 чисел)
        triangle_nodes = mesh.p[:, node_indices] # массив координат из трёх вершин (массив 2x3)
        triangle_nodes_y = triangle_nodes[1, :]  # y-координаты трёх вершин (массив из 3 чисел)
        if (np.min(triangle_nodes_y) <= 0.6) and (np.max(triangle_nodes_y) >= 0.6):
            elements.append(i)

    mesh = mesh.refined(elements)

mesh.draw()